In [1]:
import requests, bs4, time
import pandas as pd
import os.path
from datetime import datetime

In [2]:
path = os.getcwd()
parent_folder, current_folder = os.path.split(path)

# Scrape all links to job postings containing the word 'Data'
First searching for all job postings using the word 'Data' on www.monster.co.uk

In [17]:
url = 'https://www.monster.co.uk/jobs/search/?q=data&saltyp=1&cy=uk&stpage=1&page=10'

In [18]:
#  GET request
res = requests.get(url)
soup = bs4.BeautifulSoup(res.content)

Extracting all the links to the individual job postings. The links that contain 'job-openings' are only working, so select only those links:

In [19]:
def extract_links(soup):
    links =[]
    for div in soup.find_all(name='a', attrs={'data-bypass':'true'}):
        links.append(div['href'])
    return links

all_links = extract_links(soup)
links = [s for s in all_links if "job-openings" in s]

Write extracted links into a txt-File.

In [20]:
today = datetime.now().strftime('%Y-%m-%d-%H-%M')
with open(parent_folder+'/data/monster_links_'+today+'.txt', 'w') as f:
    for item in links:
        f.write("%s\n" % item)

# Scraping websites for each individual job posting link:
First extracting the links from the text file.

In [4]:
#links = [line.rstrip('\n') for line in open(parent_folder+'/data/monster_links_1.txt','r')]

Define needed functions to extract information from each website:

In [21]:
# Function for extracting the job title from the website:
def extract_jobtitle(soup):
    title = 'no_title'
    for div in soup.find_all(name='h1', attrs={'class':'title'}):
        title = div.text
    return title

# Function for extracting the job description from the website:
def extract_descr(soup):
    jobdesc = []
    for div in soup.find_all(name='span', attrs={'id':'TrackingJobBody'}):
        jobdesc = div.text
    return jobdesc

# Function for extracting salary from the website:
def extract_salary(soup):
    salary = []
    for div in soup.find_all('div',{'class':'col-xs-12 cell'}):
        salary = div.text
        salary = salary.replace('Salary','').strip()
    return salary

# Function for extracting meta-data from the website:
def extract_summary(soup):
    output = ['nan','nan','nan','nan','nan','nan','nan']
    separation =['Location', 'Job type', 'Posted', 'Industries', 'Education level', 'Career level','Reference code']
    for div in soup.find_all('dl',{'class':'header'}):
        summary = div.text
        summary = summary.replace('\n','')
        for idx, item in enumerate(separation):
            if item in summary:
                output[idx] = summary.replace(item,'')
    return output

In [22]:
job_title_company = []
description = []
salary = []
location = []
jobtype = []
industry = []
education = []
career = []
duration = []
ref_code = []

for i in range(0,len(links)):
    url = links[i]
    res = requests.get(url)
    time.sleep(1)  #ensuring at least 1 second between page grabs
    soup = bs4.BeautifulSoup(res.content)
    job_title_company.append(extract_jobtitle(soup))
    description.append(extract_descr(soup))
    salary.append(extract_salary(soup))
    location_new, jobtype_new, duration_new, industry_new, education_new, career_new, ref_code_new = extract_summary(soup)
    location.append(location_new)
    jobtype.append(jobtype_new)
    duration.append(duration_new)
    industry.append(industry_new)
    education.append(education_new)
    career.append(career_new)
    ref_code.append(ref_code_new)
    
d = {'title':job_title_company,'description':description, 'salary':salary, 'location':location, 'jobtype':jobtype, 'duration':duration, 'industry':industry, 'education':education, 'career':career,'ref_code':ref_code}
df = pd.DataFrame(d)
df.to_csv(parent_folder+'/data/monster_links_'+today+'.csv', index=True, sep='\t')

In [16]:
df

,title,description,salary,location,jobtype,duration,industry,education,career,ref_code
0,Data Incident Analysis Specialist - Nationwide,Job descriptionWe need:• Data Incident Analysi...,[],"Swindon Head Office, SW","Part Time, Permanent",Today,Other/Not Classified,nan,nan,mp25410ct1240-r002801427
1,Data Entry Clerk - Extra Personnel,Data Entry ClerkExtra Personnel are pleased to...,£9.00 - £9.00 per hour,"Banbury, HC OX164RS","Full Time, Permanent",Today,Manufacturing - Other,Unspecified,Student (Higher education/Graduate),dataen012_1571217128
2,Data Engineer - Nigel Frank International,"Data EngineerSouth London / SurreyUp to £70,00...",[],"Croydon, HC CR01HQ","Full Time, Permanent",Today,Internet Services,Bachelor's Degree,nan,cd/bd/es_1570039238
3,Senior Data Scientist / Consultant - DataTech ...,Senior Data Scientist / Consultant - £Competit...,Benefits:,"London, London","Full Time, Permanent",Today,Business Services - Other,nan,nan,j10882
4,CRM Manager – Customer Relationship Data Manag...,Customer Relationship Data Manager/ CRM Manage...,"£28,782.00 - £35,322.00 per year+ Benefits","Glasgow, Scotland G2 2HG","Full Time, Permanent",Today,All,nan,nan,awdo-p5093
...,...,...,...,...,...,...,...,...,...,...
265,Big Data engineer - TOMGANDHI CONSULTING LIMITED,Technical SkillsExperience with java and big ...,fr.450.00 - fr.500.00 /per day,"London, London","Full Time, Temporary/Contract/Project",1 Days ago,nan,nan,nan,nan
266,Engineering Data Analyst - ROOT2 RECRUITMENT L...,ENGINEERING DATA ANALYSTTEESPORT£Excellent Hou...,[],"Teesport, Yorkshire TS67SA","Full Time, Permanent",6 Days ago,Engineering Services,Unspecified,Experienced (Non-Manager),1773r2r
267,Big Data Engineer - McGregor Boyall,"Type - PermSalary- £70,000-£120,000Location - ...","£70,000.00 - £120,000.00 per year","London, London","Full Time, Permanent",2 Days ago,Other/Not Classified,Bachelor's Degree,Experienced (Non-Manager),bbbh146146_1570466717
268,no_title,[],[],nan,nan,nan,nan,nan,nan,nan
